In [ ]:
TMVA::Tools::Instance();

TFile *input             = TFile::Open("higgs.root");
TMVA::DataLoader *loader = new TMVA::DataLoader("higgs");

TString outfilename      = "TMVA_DNN_GPU.root";
TFile *output            = TFile::Open(outfilename, "RECREATE");
TMVA::Factory *factory   = new TMVA::Factory("TMVAClassification", output, "AnalysisType=Classification");

In [ ]:
TTree *tsignal     = (TTree *) input->Get("TreeS");
TTree *tbackground = (TTree *) input->Get("TreeB");
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;
loader->AddSignalTree    (tsignal,     signalWeight);
loader->AddBackgroundTree(tbackground, backgroundWeight);
loader->AddVariable("lepton_pT",'F');
loader->AddVariable("lepton_eta",'F');
loader->AddVariable("lepton_phi",'F');
loader->AddVariable("missing_energy_magnitude",'F');
loader->AddVariable("missing_energy_phi",'F');
loader->AddVariable("jet_1_pt",'F');
loader->AddVariable("jet_1_eta",'F');
loader->AddVariable("jet_1_phi",'F');
loader->AddVariable("jet_1_b_tag",'F');
loader->AddVariable("jet_2_pt",'F');
loader->AddVariable("jet_2_eta",'F');
loader->AddVariable("jet_2_phi",'F');
loader->AddVariable("jet_2_b_tag",'F');
loader->AddVariable("jet_3_pt",'F');
loader->AddVariable("jet_3_eta",'F');
loader->AddVariable("jet_3_phi",'F');
loader->AddVariable("jet_3_b_tag",'F');
loader->AddVariable("jet_4_pt",'F');
loader->AddVariable("jet_4_eta",'F');
loader->AddVariable("jet_4_phi",'F');
loader->AddVariable("jet_4_b_tag",'F');
loader->AddVariable("m_jj",'F');
loader->AddVariable("m_jjj",'F');
loader->AddVariable("m_lv",'F');
loader->AddVariable("m_jlv",'F');
loader->AddVariable("m_bb",'F');
loader->AddVariable("m_wbb",'F');
loader->AddVariable("m_wwbb",'F');

In [ ]:
TString dataString = "nTrain_Signal=5000000:"
                     "nTrain_Background=5000000:"
                     "nTest_Signal=100000:"
                     "nTest_Background=100000:"
                     "SplitMode=Random:"
                     "NormMode=NumEvents:"
                     "!V";
loader->PrepareTrainingAndTestTree("", "", dataString);-

In [ ]:
factory->BookMethod(loader, TMVA::Types::kBDT, "BDT", "!V:NTrees=1000:MinNodeSize=2.5%:MaxDepth=3"
                                                      ":BoostType=AdaBoost:AdaBoostBeta=0.5"
                                                      ":UseBaggedBoost:BaggedSampleFraction=0.5"
                                                      ":SeparationType=GiniIndex:nCuts=20");

In [ ]:
TString configString = "!H:V";
configString += ":Architecture=GPU";
configString += ":VarTransform=N";
configString += ":ErrorStrategy=CROSSENTROPY";
configString += ":WeightInitialization=XAVIERUNIFORM";
TString layoutString = "Layout=TANH|256,TANH|256,TANH|256,TANH|256,TANH|256,LINEAR";

TString trainingPhase1 = "TrainingStrategy="
                         "LearningRate=0.1,"
                         "ConvergenceSteps=20,"
                         "Momentum=0.9,"
                         "BatchSize=4096,"
                         "Regularization=L2,"
                         "WeightDecay=1e-5,"
                         "TestRepetitions=5";

configString += ":" + layoutString;
configString += ":" + trainingPhase1;
TMVA::Types::EMVA method = TMVA::Types::kDNN;
factory->BookMethod(loader, method, "DNN", configString);

In [ ]:
factory->TrainAllMethods();
factory->TestAllMethods();
factory->EvaluateAllMethods();

In [ ]:
%jsroot on
TCanvas *c=factory->GetROCCurve(loader);
c->Draw();